In [1]:
from sumolib import checkBinary  # noqa
import sumolib
import traci  # noqa
import os
import sys
from pathlib import Path
import traci

try:
    dir = Path("Simulation_Environment\Main DQN")
    sumoBinary = checkBinary('sumo')
    # sumoBinary = checkBinary('sumo-gui')
    # we need to import python modules from the $SUMO_HOME/tools directory
    if 'SUMO_HOME' in os.environ:
        tools = os.path.join(os.environ['SUMO_HOME'], 'tools')
        sys.path.append(tools)
    else:
        sys.exit("please declare environment variable 'SUMO_HOME'")
    traci.start([sumoBinary, "-c", f"{dir}\osm.sumocfg",
                            "--tripinfo-output", f"{dir}\\Results\\tripinfo.xml",  "--start"])
    #start net
    net = sumolib.net.readNet(Path(f"{dir}\\osm.net.xml"))
except Exception as e:
    print(e)

print("Traci and sumolib Net is running in background...")


Traci and sumolib Net is running in background...


In [5]:
net = sumolib.net.readNet(Path(f"{dir}\\osm.net.xml"))


In [6]:
traci.trafficlight.getIDList()

('corrales_recto_tl',
 'luna_corrales_tl',
 'luna_roa_tl',
 'osmena_florentino_tl',
 'recto_osmena_tl')

In [20]:

phases = traci.trafficlight.getCompleteRedYellowGreenDefinition('recto_osmena_tl')[0].getPhases()
min_val = min([i.duration for i in phases])
duration = [i.duration for i in phases if 'y' not in i.state]
duration

[33.0, 33.0, 33.0]

In [3]:
import numpy as np
x = np.round(np.hstack([[1,2,3],[4,5,6]])/5)
x

array([0., 0., 1., 1., 1., 1.])

In [26]:
tls =  'recto_osmena_tl'
traci.trafficlight.getRedYellowGreenState(tls)
traci.trafficlight.getCompleteRedYellowGreenDefinition(tls)[0].getPhases()[0].duration = 34
traci.trafficlight.getCompleteRedYellowGreenDefinition(tls)[0].getPhase()

AttributeError: 'Logic' object has no attribute 'getPhase'

In [80]:
all_nodes = net.getNodes()
tls_nodes = [node for node in all_nodes if node.getType() == 'traffic_light']
tls_nodes[0]

In [81]:
x = tls_nodes[0]._coord

y = tls_nodes[1]._coord
print(x,y)

(1270.82, 843.94, 0.0) (1080.72, 805.03, 0.0)


In [82]:
import numpy as np

def distance(x,y):
    x_a = x[0]
    x_b = x[1]
    y_a = y[0]
    y_b = y[1]
    c = np.sqrt(((x_a-y_a)**2)+((x_b-y_b)**2))
    return c

In [83]:
distance(x,y)

194.04122783573592

In [84]:
import pandas as pd
array = np.zeros([len(tls_nodes)]*2)
column_names = [i.getID() for i in tls_nodes]
df = pd.DataFrame(array, columns=column_names, index=column_names)
for row in column_names:
    for column in column_names:
        row_node = net.getNode(row)
        column_node = net.getNode(column)
        row_coords = row_node._coord
        column_coords = column_node._coord
        df.loc[row,column] = distance(row_coords,column_coords)

df
        

,osmena_florentino,luna_roa,luna_corrales,recto_osmena,corrales_recto
osmena_florentino,0.000000,194.041228,429.608674,243.748077,434.019620
luna_roa,194.041228,0.000000,242.217345,365.066097,368.590317
luna_corrales,429.608674,242.217345,0.000000,533.873979,340.458992
recto_osmena,243.748077,365.066097,533.873979,0.000000,355.016847
corrales_recto,434.019620,368.590317,340.458992,355.016847,0.000000


In [85]:
list(df['osmena_florentino'].sort_values()[1:3].index)

['luna_roa', 'recto_osmena']

In [86]:
x = net.getTLS('recto_osmena_tl')
x.getID()


'recto_osmena_tl'

In [88]:
traci.trafficlight.getCompleteRedYellowGreenDefinition('recto_osmena_tl')[0].getPhases()

(Phase(duration=33.0, state='rrrrrGGGggrrrrGGGr', minDur=33.0, maxDur=33.0, next=()),
 Phase(duration=6.0, state='rrrrryyyggrrrryyyr', minDur=6.0, maxDur=6.0, next=()),
 Phase(duration=6.0, state='rrrrrrrrGGrrrrrrrG', minDur=6.0, maxDur=6.0, next=()),
 Phase(duration=6.0, state='rrrrrrrryyrrrrrrry', minDur=6.0, maxDur=6.0, next=()),
 Phase(duration=33.0, state='GGGggrrrrrGGGgrrrr', minDur=33.0, maxDur=33.0, next=()),
 Phase(duration=6.0, state='yyyyyrrrrryyyyrrrr', minDur=6.0, maxDur=6.0, next=()))

In [58]:
'recto_osmena'+'_tl'

'recto_osmena_tl'

In [22]:
z = tls_nodes[3]
x = z.getNeighboringNodes()
[(i.getType(),i.getID()) for i in x]

[('dead_end', 'east_recto'),
 ('dead_end', 'north_osmena'),
 ('traffic_light', 'osmena_florentino'),
 ('priority', 'roa_recto'),
 ('traffic_light', 'corrales_recto')]

In [60]:
def get_neighbors(neighbour_limit,net):
        """Returns the neighbouring nodes"""
        def distance(x,y):  #Distance between two points in a cartesea plane (No Elevation)
            x_a = x[0]
            x_b = x[1]
            y_a = y[0]
            y_b = y[1]
            c = np.sqrt(((x_a-y_a)**2)+((x_b-y_b)**2))
            return c

        all_nodes = net.getNodes()
        tls_nodes = [node for node in all_nodes if node.getType() == 'traffic_light']
        array = np.zeros([len(tls_nodes)]*2)
        column_names = [i.getID() for i in tls_nodes]
        df = pd.DataFrame(array, columns=column_names, index=column_names)
        for row in column_names:
            for column in column_names:
                row_node = net.getNode(row)
                column_node = net.getNode(column)
                row_coords = row_node._coord
                column_coords = column_node._coord
                df.loc[row,column] = distance(row_coords,column_coords)
        distance_dict = {}
        for node in column_names:
            node_tl_name = node+'_tl'
            lst=[i+'_tl' for i in list(df[node].sort_values()[1:neighbour_limit+1].index)]
            distance_dict[node_tl_name] = lst
        return distance_dict

get_neighbors(2,net)

{'osmena_florentino_tl': ['luna_roa_tl', 'recto_osmena_tl'],
 'luna_roa_tl': ['osmena_florentino_tl', 'luna_corrales_tl'],
 'luna_corrales_tl': ['luna_roa_tl', 'corrales_recto_tl'],
 'recto_osmena_tl': ['osmena_florentino_tl', 'corrales_recto_tl'],
 'corrales_recto_tl': ['luna_corrales_tl', 'recto_osmena_tl']}

In [65]:
from IPython.display import display
dis_1 = traci.trafficlight.getIDList()
dis_2 = [i.getID()+'_tl' for i in tls_nodes]
for i in dis_1:
    if len(dis_1) != len(dis_2):
        print('Lacking TLS')
        break
    if i in dis_2:
        print(f'{i}:    /')
    else:
        print(f'{i}:    X')

corrales_recto_tl:    /
luna_corrales_tl:    /
luna_roa_tl:    /
osmena_florentino_tl:    /
recto_osmena_tl:    /


In [ ]:
x = net.getTrafficLights()[1]
print(x.__dir__())

In [69]:
x_list = [1,2,3,4,5]
y_list = [1,2,3,8,7]

empty_list = []

for i in x_list:
    if i not in y_list:
        empty_list.append(i)

for i in y_list:
    if i not in x_list:
        empty_list.append(i)

empty_list

[4, 5, 8, 7]

In [76]:
traci.close()